In [1]:
# Import the packages necessary for this code
import geopandas as gpd
import pandas as pd

In [2]:
# Open the National Speed Limit Registrar
# The NSLR will be used to select case studies
NSLR = gpd.read_file(f'NSLR_NZ/SpeedLimitZoneFull.shp')

C:\Users\caleb\anaconda3\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: NSLR_NZ/SpeedLimitZoneFull.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [9]:
# Observe NSLR structure
print(NSLR.head(n=1))

print(NSLR.columns)

# Observe how variable speed limits are stored in the NSLR
print(NSLR['speedCateg'].unique())

# Observe how schools are stored in the NSLR
print(NSLR['speedLim_6'].unique())

                             speedLimit                            speedLim_1  \
0  fbe4d1a4-623f-4f9a-8bdf-767dc93d30d9  52e6cbbb-71ff-4cc8-ae5b-ab102b437ee9   

                             rcaZoneRef       rcaZoneR_1  \
0  21985b18-e222-4644-a004-28081ef25287  Buller District   

                                       legalRefer  \
0  Buller District Council Speed Limit Bylaw 2007   

                                       legalInstr legalIns_1 speedManag  \
0  Buller District Council Speed Limit Bylaw 2007       None       None   

  speedMan_1 speedCateg  ... speedLi_10 speedLi_11 speedLi_12 whenEffect  \
0       None  Permanent  ...       None 1900-01-01 1899-12-30 1900-01-01   

  whenIneffe                              GlobalID SeperateLa  speedLi_13  \
0 1899-12-30  3e7bd2c0-388d-4508-8ebe-95f611143191       None        None   

  speedLi_14                                           geometry  
0  MOKIHINUI  POLYGON ((1511362.189 5402116.332, 1511410.803...  

[1 rows x 28 colu

In [4]:
# Subset NSLR to only include variable speed limits in school zones
NSLR_var_school = NSLR[(NSLR['speedCateg']=="Variable") & (NSLR['speedLim_6']=="The presence of a school")]

print(f"{round(100*len(NSLR_var_school)/len(NSLR),2)}% of road segements as schools with a varaible speed limit")

3.87% of road segements as schools with a varaible speed limit


In [5]:
# Define start and end data of GPS data
start_date = pd.Timestamp('2019-02-01')
end_date = pd.Timestamp('2023-03-01')

# Filter NSLR to only include schools with var speeds implemented during GPS time period
NSLR_filt_hard = NSLR_var_school[(pd.to_datetime(NSLR['whenEffect']) < end_date) & (pd.to_datetime(NSLR['whenEffect']) > start_date)]

# Filter NSLR_filt_hard to 
NSLR_filt_hard = NSLR_filt_hard.assign(
    speedLi_14_trim=NSLR_filt_hard['speedLi_14'].astype(str).apply(lambda x: ' '.join(x.split(' ')[:2])))
NSLR_final = NSLR_filt_hard.drop_duplicates(subset='speedLi_14_trim')

NSLR_final.loc[:, 'VarDurGPS'] = 1

print(f'There are {len(NSLR_final)} schools which implemented variable speed limits within the GPS time period')

# Observe # of VSLs in school zones by area in New Zealand
# Case studies will be selected to be within a specific area
NSLR_final[['VarDurGPS', 'rcaZoneR_1']].groupby(['rcaZoneR_1']).describe()

There are 32 schools which implemented variable speed limits within the GPS time period


C:\Users\caleb\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1896: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\caleb\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


VarDurGPS                                   
                            count mean  std  min  25%  50%  75%  max
rcaZoneR_1                                                          
Auckland                     10.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Hamilton City                 1.0  1.0  NaN  1.0  1.0  1.0  1.0  1.0
Invercargill City             4.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Masterton District            6.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Matamata-Piako District       2.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
State Highways                6.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Waikato District              1.0  1.0  NaN  1.0  1.0  1.0  1.0  1.0
Whanganui District            2.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0

In [8]:
# Auckland, Invercargill, State Highways, and Masterton have enough schools with VSLs for case studies
# Want to minimise different times VSLs were implemented
locations = NSLR_final['rcaZoneR_1'].unique().tolist()

# Find number of dates VSLs went into effect by region
for location in locations:
    NSLR_locs = NSLR_final[NSLR_final['rcaZoneR_1']==location]
    print(f"{location} implemented {len(NSLR_locs)} VSLs on {NSLR_locs['whenEffect'].nunique()} different times")
    print()

Matamata-Piako District implemented 2 VSLs on 1 different times

Masterton District implemented 6 VSLs on 1 different times

Whanganui District implemented 2 VSLs on 1 different times

State Highways implemented 6 VSLs on 5 different times

Hamilton City implemented 1 VSLs on 1 different times

Auckland implemented 10 VSLs on 3 different times

Waikato District implemented 1 VSLs on 1 different times

Invercargill City implemented 4 VSLs on 1 different times



In [7]:
# Masterton District is the best choice for case studies as it 
#implemented variable speed limits in six schools all on the same date.
# Subset based on Masterton District 
case_studies_hard = NSLR_filt_hard[NSLR_filt_hard['rcaZoneR_1']=='Masterton District']

# Save final case study locations
case_studies_hard.to_csv(f'Case_Studies_Locations.csv')